In [13]:
# ========== AKShare工具函数 ==========
import akshare as ak
import pandas as pd
from typing import Optional
import logging

# 配置日志 - 使用WARNING级别避免刷屏
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

# 强制关闭干扰日志（确保流式输出纯净）
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("sentence_transformers").setLevel(logging.WARNING)
logging.getLogger("chromadb").setLevel(logging.WARNING)
logging.getLogger("akshare").setLevel(logging.WARNING)
logging.getLogger("src.analyst_chain.tools.akshare_tools").setLevel(logging.WARNING)


def get_gdp_quarterly() -> str:
    """获取季度GDP数据

    Returns:
        格式化的GDP数据字符串
    """
    print("[工具] 获取GDP数据...", end="", flush=True)
    try:
        df = ak.macro_china_gdp()
        if df.empty:
            print(" 失败")
            return "获取GDP数据失败：数据为空"

        df = df.rename(columns={
            '季度': 'quarter',
            '国内生产总值-绝对值': 'gdp',
            '国内生产总值-同比增长': 'gdp_yoy'
        })
        df = df[['quarter', 'gdp', 'gdp_yoy']]
        df['gdp'] = pd.to_numeric(df['gdp'], errors='coerce')
        df['gdp_yoy'] = pd.to_numeric(df['gdp_yoy'], errors='coerce')
        df = df.dropna().reset_index(drop=True)

        result = "GDP季度数据(最近5个季度):\n\n"
        for _, row in df.head(5).iterrows():
            result += f"季度: {row['quarter']}, GDP: {row['gdp']}亿元, 同比增长: {row['gdp_yoy']}%\n"

        print(" 完成")
        return result
    except Exception as e:
        print(f" 失败: {e}")
        return f"获取GDP数据失败: {str(e)}"


def get_cpi_monthly() -> str:
    """获取月度CPI数据

    Returns:
        格式化的CPI数据字符串
    """
    print("[工具] 获取CPI数据...", end="", flush=True)
    try:
        df = ak.macro_china_cpi()
        if df.empty:
            print(" 失败")
            return "获取CPI数据失败：数据为空"

        df = df.rename(columns={
            '月份': 'month',
            '全国-同比增长': 'cpi_yoy',
            '全国-环比增长': 'cpi_mom',
            '全国-累计': 'cpi_ytd'
        })
        df = df[['month', 'cpi_yoy', 'cpi_mom', 'cpi_ytd']]
        df['cpi_yoy'] = pd.to_numeric(df['cpi_yoy'], errors='coerce')
        df['cpi_mom'] = pd.to_numeric(df['cpi_mom'], errors='coerce')
        df['cpi_ytd'] = pd.to_numeric(df['cpi_ytd'], errors='coerce')
        df = df.dropna().reset_index(drop=True)

        result = "CPI月度数据(最近6个月):\n\n"
        for _, row in df.head(6).iterrows():
            result += f"月份: {row['month']}, 同比: {row['cpi_yoy']}%, 环比: {row['cpi_mom']}%, 累计: {row['cpi_ytd']}%\n"

        print(" 完成")
        return result
    except Exception as e:
        print(f" 失败: {e}")
        return f"获取CPI数据失败: {str(e)}"


def get_pmi_manufacturing() -> str:
    """获取制造业PMI数据

    Returns:
        格式化的PMI数据字符串
    """
    print("[工具] 获取PMI数据...", end="", flush=True)
    try:
        df = ak.macro_china_pmi()
        if df.empty:
            print(" 失败")
            return "获取PMI数据失败：数据为空"

        df = df.rename(columns={
            '月份': 'month',
            '制造业-指数': 'pmi',
            '制造业-同比增长': 'pmi_yoy'
        })
        df = df[['month', 'pmi', 'pmi_yoy']]
        df['pmi'] = pd.to_numeric(df['pmi'], errors='coerce')
        df['pmi_yoy'] = pd.to_numeric(df['pmi_yoy'], errors='coerce')
        df = df.dropna().reset_index(drop=True)

        result = "PMI制造业数据(最近6个月):\n\n"
        for _, row in df.head(6).iterrows():
            result += f"月份: {row['month']}, PMI指数: {row['pmi']}, 同比增长: {row['pmi_yoy']}%\n"

        print(" 完成")
        return result
    except Exception as e:
        print(f" 失败: {e}")
        return f"获取PMI数据失败: {str(e)}"


# ========== 知识库检索工具 ==========
import json
from pathlib import Path
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings


class KnowledgeRetriever:
    """知识库检索工具"""

    def __init__(self,
                 vector_db_path: str = "../data/processed/knowledge/vector_db/knowledge_base",
                 json_dir_path: str = "../data/processed/knowledge/structured/knowledge_base",
                 embedding_model: Optional[str] = None):
        """初始化知识库检索器"""
        self.vector_db_path = Path(vector_db_path)
        self.json_dir_path = Path(json_dir_path)

        if embedding_model is None:
            import os
            embedding_model = os.getenv('EMBEDDING_MODEL_PATH', 'Qwen/Qwen3-Embedding-0.6B')

        self.embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
        self.vector_store = Chroma(
            collection_name="knowledge_base_col",
            persist_directory=str(self.vector_db_path),
            embedding_function=self.embeddings
        )

        self._load_json_index()

    def _load_json_index(self):
        """加载JSON文件索引"""
        self.json_files = {}
        for json_file in self.json_dir_path.glob("*.json"):
            topic_num = int(json_file.name.split("_")[0])
            self.json_files[topic_num] = json_file

    def vector_search(self, query: str, k: int = 3) -> str:
        """向量检索"""
        results = self.vector_store.similarity_search(query, k=k)

        if not results:
            return "未找到相关知识"

        output = f"向量检索结果(共{len(results)}条):\n\n"
        for i, doc in enumerate(results, 1):
            output += f"[结果{i}]\n"
            output += f"内容: {doc.page_content[:200]}...\n"
            if doc.metadata:
                output += f"来源: 主题{doc.metadata.get('seq', 'N/A')} - {doc.metadata.get('topic', 'N/A')}\n"
            output += "\n"

        return output

    def get_topic_knowledge(self, topic_number: int) -> str:
        """按主题查询JSON知识"""
        if topic_number not in self.json_files:
            return f"错误: 主题{topic_number}不存在(有效范围1-17)"

        json_file = self.json_files[topic_number]
        with open(json_file, 'r', encoding='utf-8') as f:
            knowledge = json.load(f)

        output = f"主题{topic_number}: {knowledge.get('topic', 'N/A')}\n\n"

        if 'key_concepts' in knowledge:
            output += "关键概念:\n"
            for concept in knowledge['key_concepts'][:5]:
                name = concept.get('name', 'N/A')
                definition = concept.get('definition', 'N/A')
                output += f"  - {name}: {definition}\n"
            output += "\n"

        if 'indicators' in knowledge:
            output += "关键指标:\n"
            for indicator in knowledge['indicators'][:3]:
                name = indicator.get('name', 'N/A')
                description = indicator.get('description', 'N/A')
                output += f"  - {name}: {description}\n"
            output += "\n"

        if 'summary' in knowledge:
            output += f"摘要:\n{knowledge['summary'][:300]}...\n"

        return output

print("[完成] 工具函数加载完成 (日志级别: WARNING)")

[完成] 工具函数加载完成 (日志级别: WARNING)


In [ ]:
# 阶段2：宏观经济SubAgent实现（DeepAgent架构）

import os
import sys

# 加载环境变量
from dotenv import load_dotenv
load_dotenv('../config/.env')

from langchain_openai import ChatOpenAI
from deepagents import create_deep_agent

print("[初始化] 1/4 加载环境变量...")
sys.stdout.flush()

# 初始化DeepSeek模型
deepseek_model = ChatOpenAI(
    model="deepseek-chat",
    openai_api_key=os.getenv("DEEPSEEK_API_KEY"),
    openai_api_base="https://api.deepseek.com",
    streaming=True,
    temperature=0.7,
)

print("[初始化] 2/4 创建知识检索器...")
sys.stdout.flush()

# 初始化知识检索器
knowledge_retriever = KnowledgeRetriever()

print("[初始化] 3/4 包装知识检索工具...")
sys.stdout.flush()

# 包装知识检索方法为独立函数
def vector_search(query: str, k: int = 3) -> str:
    """向量检索知识库

    Args:
        query: 查询文本
        k: 返回结果数量(默认3)

    Returns:
        检索结果文本
    """
    print("[工具] 向量检索...", end="", flush=True)
    result = knowledge_retriever.vector_search(query, k)
    print(" 完成")
    return result

def get_topic_knowledge(topic_number: int) -> str:
    """按主题编号查询JSON知识

    Args:
        topic_number: 主题编号(1-17)

    Returns:
        主题知识内容
    """
    print(f"[工具] 查询主题{topic_number}...", end="", flush=True)
    result = knowledge_retriever.get_topic_knowledge(topic_number)
    print(" 完成")
    return result

print("[初始化] 4/4 创建DeepAgent+SubAgent...")
sys.stdout.flush()

# 定义宏观经济SubAgent配置（字典格式）
macroeconomic_subagent = {
    "name": "macroeconomic_subagent",
    "description": "负责宏观经济分析,包括GDP、CPI、PMI等指标的数据获取、分析和趋势判断。结合实时数据和理论知识提供专业分析。",
    "system_prompt": """你是宏观经济分析专家。

核心能力：
1. 数据获取：使用AKShare工具获取最新宏观经济数据(GDP、CPI、PMI等)
2. 理论支撑：查询知识库获取相关经济理论和分析框架
3. 综合分析：结合实时数据和理论知识，提供专业的经济解读

分析流程：
1. 获取相关数据（使用get_gdp_quarterly/get_cpi_monthly/get_pmi_manufacturing）
2. 查询理论框架（使用vector_search）
3. 分析数据趋势和周期特征
4. 给出专业判断和预测

输出要求：
- 数据准确，引用来源（如"根据AKShare最新数据..."）
- 分析专业，运用理论（如"根据经济周期理论..."）
- 逻辑清晰，结构完整（数据→理论→分析→结论）
- 结论明确，便于理解""",
    "tools": [
        get_gdp_quarterly,
        get_cpi_monthly,
        get_pmi_manufacturing,
        vector_search,
        get_topic_knowledge,
    ],
}

# 创建Main Agent（使用create_deep_agent）
main_agent = create_deep_agent(
    model=deepseek_model,
    subagents=[macroeconomic_subagent],
)

print("[成功] DeepAgent+SubAgent架构已创建")
print(f"   类型: {type(main_agent).__name__}")
print(f"   模型: DeepSeek (deepseek-chat)")
print(f"   SubAgent: macroeconomic_subagent")
print(f"   工具数量: 5个")
print(f"     - AKShare工具: 3个 (GDP/CPI/PMI)")
print(f"     - 知识检索: 2个 (向量检索/JSON查询)")
print("\n[提示] Token级流式输出已启用，带详细进度追踪")

[初始化] 1/4 加载环境变量...
[初始化] 2/4 创建知识检索器...


[初始化] 3/4 包装知识检索工具...
[初始化] 4/4 创建DeepAgent+SubAgent...
[成功] DeepAgent+SubAgent架构已创建
   类型: CompiledStateGraph
   模型: DeepSeek (deepseek-chat)
   SubAgent: macroeconomic_subagent
   工具数量: 5个
     - AKShare工具: 3个 (GDP/CPI/PMI)
     - 知识检索: 2个 (向量检索/JSON查询)

[提示] Token级流式输出已启用，带详细进度追踪


## 阶段2任务进度

**总进度**: 3/3任务 (100%) ✅

| 任务 | 状态 | 说明 |
|------|------|------|
| 1. 核心Tools封装 | ✅ 完成 | 知识库检索(2个) + AKShare(3个GDP/CPI/PMI) |
| 2. Main+SubAgent实现 | ✅ 完成 | DeepAgent架构，1个宏观经济SubAgent |
| 3. 测试与验证 | 🔄 进行中 | 本notebook正在验证DeepAgent架构性能 |

**当前任务**: 测试3 - 验证DeepAgent+SubAgent架构的执行流程和性能

**测试目标**:
- 验证Main Agent → SubAgent的调用流程
- 追踪180秒延迟的详细过程
- 确认工具调用和知识检索的正确性

---

## 测试场景

测试宏观经济SubAgent的综合分析能力（DeepAgent架构）

In [15]:
# 测试1：GDP分析（DeepAgent详细进度追踪）
import sys
import time
from datetime import datetime
from langchain_core.messages import AIMessageChunk, ToolMessage

test_query_1 = "最近GDP增长率如何？有什么趋势特征？"

# 进度追踪
print("=" * 80)
print(f"[总进度] 测试1/3: GDP分析 (DeepAgent+SubAgent架构)")
print("=" * 80)
print(f"问题：{test_query_1}")
print("-" * 80)

start_time = time.time()
print(f"\n[{datetime.now().strftime('%H:%M:%S')}] 开始执行")
print("[进度] 1/6 发送问题到Main Agent...")
sys.stdout.flush()

# 状态追踪
stage_times = {}
tool_call_count = 0
output_started = False
first_token_time = None
subagent_started = False

print("[进度] 2/6 Main Agent决策中（分析问题，选择SubAgent）...")
sys.stdout.flush()

# 记录Main Agent决策时间
stage_times['main_agent_start'] = time.time()

# LangGraph的stream方法（字典输入 + stream_mode="messages"）
response_text = ""
for message_chunk, metadata in main_agent.stream(
    {"messages": [{"role": "user", "content": test_query_1}]},
    stream_mode="messages"
):
    current_time = time.time()

    # 检测工具调用（SubAgent执行）
    if isinstance(message_chunk, ToolMessage):
        if not subagent_started:
            elapsed = current_time - start_time
            print(f"\n[{datetime.now().strftime('%H:%M:%S')}] [进度] 3/6 Main Agent已决策，调用SubAgent (耗时{elapsed:.1f}s)")
            print("[进度] 4/6 SubAgent执行中（调用工具+分析）...")
            sys.stdout.flush()
            subagent_started = True
        tool_call_count += 1
        elapsed = current_time - start_time
        print(f"\n[{datetime.now().strftime('%H:%M:%S')}] 工具调用 {tool_call_count} 完成 (耗时{elapsed:.1f}s)")
        sys.stdout.flush()

    # 检测AI消息（流式输出）
    elif isinstance(message_chunk, AIMessageChunk):
        # 检测首个token返回
        if message_chunk.content and not output_started:
            first_token_time = current_time - start_time
            print(f"\n[{datetime.now().strftime('%H:%M:%S')}] [进度] 5/6 SubAgent返回结果 (总延迟{first_token_time:.1f}s)")
            print("[进度] 6/6 流式输出中...")
            print(f"\n回答：", end="")
            output_started = True
            sys.stdout.flush()

        # 输出流式内容
        if message_chunk.content:
            print(message_chunk.content, end="")
            sys.stdout.flush()
            response_text += message_chunk.content

total_time = time.time() - start_time
print(f"\n\n[{datetime.now().strftime('%H:%M:%S')}] [完成]")
print("-" * 80)
print(f"[统计] 总耗时: {total_time:.2f}s | 工具调用: {tool_call_count}次 | 首token延迟: {first_token_time:.1f}s")
print("=" * 80)

[总进度] 测试1/3: GDP分析 (DeepAgent+SubAgent架构)
问题：最近GDP增长率如何？有什么趋势特征？
--------------------------------------------------------------------------------

[10:55:23] 开始执行
[进度] 1/6 发送问题到Main Agent...
[进度] 2/6 Main Agent决策中（分析问题，选择SubAgent）...



[10:55:25] [进度] 5/6 SubAgent返回结果 (总延迟1.8s)
[进度] 6/6 流式输出中...

回答：我来为您分析最近的GDP增长率和趋势特征。让我使用宏观经济分析子代理来获取最新的数据和专业分析。[工具] 获取GDP数据... 完成
[工具] 向量检索... 完成
[工具] 向量检索... 完成
[工具] 向量检索... 完成
[工具] 查询主题1... 完成
[工具] 获取CPI数据... 完成
[工具] 获取PMI数据... 完成

[10:57:56] [进度] 3/6 Main Agent已决策，调用SubAgent (耗时152.1s)
[进度] 4/6 SubAgent执行中（调用工具+分析）...

[10:57:56] 工具调用 1 完成 (耗时152.1s)
根据宏观经济分析子代理的专业分析，以下是关于最近GDP增长率和趋势特征的详细总结：

## 📊 **最新GDP增长率数据**

### **近期表现**
- **2025年前三季度**：GDP增长**5.2%**，高于2024年同期的4.8%
- **季度节奏**：呈现"前高后稳"特征
  - 2025年一季度：5.4%
  - 2025年二、三季度：稳定在5.2-5.3%区间
- **2024年全年**：增长**5.0%**
- **2025年预期**：有望实现5.1-5.3%的增长

## 📈 **主要趋势特征**

### **1. 温和复苏态势**
- 经济从2024年的4.8%增长回升至2025年的5.2%
- 显示经济正在温和复苏，但增速相对平稳

### **2. 增长结构转型**
- **消费主导型特征明显**：消费占比约53%，成为经济增长的主要驱动力
- **投资支撑作用**：投资占比约44%，继续发挥重要支撑作用
- **净出口贡献有限**：仅占约3%

### **3. 经济周期位置**
当前中国经济处于**复苏阶段向稳定增长过渡期**，表现为：
- ✅ **产出增长稳定**：GDP增长率保持在5%以上
- ⚠️ **通胀压力温和**：CPI在0-1%区间，显示需求相对平稳
- ⚠️ **制造业仍在调整**：PMI略低于荣枯线，制造业面临一定压力

## 🔍 **关键影响因素**

### **积极因素**
1. **消费持续恢复**：内

In [16]:
# 测试2：通胀分析（带进度提示）
import sys
import time
from langchain_core.messages import AIMessageChunk, ToolMessage

test_query_2 = "当前通胀水平怎么样？根据经济理论应该如何解读？"
print(f"问题：{test_query_2}")
print("-" * 60)
print("\n[Agent分析中...]", end="")
sys.stdout.flush()

tool_call_count = 0
output_started = False

for message_chunk, metadata in agent.stream(
    {"messages": [{"role": "user", "content": test_query_2}]},
    stream_mode="messages"
):
    if isinstance(message_chunk, ToolMessage):
        tool_call_count += 1
        print(f"\r[工具调用 {tool_call_count}/5]", end="")
        sys.stdout.flush()

    elif isinstance(message_chunk, AIMessageChunk) and message_chunk.content:
        if not output_started:
            print("\r" + " " * 40)
            print("\n回答：", end="")
            output_started = True

        print(message_chunk.content, end="")
        sys.stdout.flush()

print("\n")

问题：当前通胀水平怎么样？根据经济理论应该如何解读？
------------------------------------------------------------

                                        

回答：我来帮您分析当前的通胀情况。首先让我获取最新的GDP数据，这通常与通胀水平有密切关系。[工具] 获取GDP数据... 完成
[工具调用 1/5]基于最新的GDP数据，我可以为您分析通胀情况：

## 当前经济状况分析

从GDP数据来看：
1. **经济增长稳定**：2025年前三季度GDP同比增长5.2%，保持在中高速增长区间
2. **增长态势良好**：从2024年的4.8%增长到2025年的5.2%，显示经济复苏势头
3. **季度表现稳健**：各季度GDP增速在5.0%-5.4%之间，波动较小

## 通胀水平的经济理论解读

### 1. **菲利普斯曲线关系**
根据经典的菲利普斯曲线理论，经济增长与通胀存在正向关系：
- 当前5.2%的经济增长可能对应温和的通胀水平
- 经济增长稳定但不过热，通胀压力可能相对可控

### 2. **产出缺口分析**
- 实际GDP增速高于潜在增长率时，会产生正向产出缺口，推高通胀
- 当前5.2%的增速如果接近潜在增长率，通胀压力可能较小

### 3. **需求拉动型通胀**
- 经济增长主要由消费和投资拉动时，可能产生需求拉动型通胀
- 需要结合消费价格指数(CPI)和生产者价格指数(PPI)具体分析

### 4. **成本推动因素**
- 原材料价格上涨、劳动力成本上升等可能带来成本推动型通胀
- 这在经济复苏阶段较为常见

## 政策建议

基于当前经济数据：
1. **货币政策**：如果通胀温和，可维持相对宽松的货币政策支持经济增长
2. **财政政策**：可继续实施积极的财政政策，但要关注通胀预期管理
3. **结构性改革**：通过供给侧改革缓解成本推动型通胀压力

**注意**：要准确评估通胀水平，还需要结合CPI、PPI、核心通胀率等具体价格指数数据。当前GDP数据显示经济稳健增长，这通常对应温和的通胀环境，但具体通胀水平需要查看价格指数数据才能精确判断。



In [17]:
# 测试3：PMI经济趋势分析（带进度提示）
import sys
import time
from langchain_core.messages import AIMessageChunk, ToolMessage

test_query_3 = "PMI数据显示经济趋势如何？结合经济周期理论分析"
print(f"问题：{test_query_3}")
print("-" * 60)
print("\n[Agent分析中...]", end="")
sys.stdout.flush()

tool_call_count = 0
output_started = False

for message_chunk, metadata in agent.stream(
    {"messages": [{"role": "user", "content": test_query_3}]},
    stream_mode="messages"
):
    if isinstance(message_chunk, ToolMessage):
        tool_call_count += 1
        print(f"\r[工具调用 {tool_call_count}/5]", end="")
        sys.stdout.flush()

    elif isinstance(message_chunk, AIMessageChunk) and message_chunk.content:
        if not output_started:
            print("\r" + " " * 40)
            print("\n回答：", end="")
            output_started = True

        print(message_chunk.content, end="")
        sys.stdout.flush()

print("\n")

问题：PMI数据显示经济趋势如何？结合经济周期理论分析
------------------------------------------------------------

                                        

回答：我主要专注于GDP数据分析，目前无法直接获取PMI数据来分析经济趋势。不过，我可以先为您提供最新的GDP数据，然后基于经济周期理论来分析整体经济状况。

让我获取最新的季度GDP数据：[工具] 获取GDP数据... 完成
[工具调用 1/5]基于最新的GDP数据，我可以从经济周期理论的角度为您分析当前的经济趋势：

## GDP数据分析

从数据可以看出：
1. **2024年全年增长5.0%**，经济保持稳定增长
2. **2025年第一季度同比增长5.4%**，显示出较强的增长势头
3. **2025年前三季度累计增长5.2%**，增长略有放缓但仍保持稳健

## 经济周期理论分析

### 1. **当前周期位置**
- 从GDP增速看（5.0%-5.4%），经济处于**扩张期**或**繁荣期**的早期阶段
- 增速相对稳定，没有出现明显的过热迹象

### 2. **周期特征**
- **扩张特征**：连续多个季度保持5%以上的增长
- **稳定性**：增速波动较小（4.8%-5.4%），显示出较好的经济韧性
- **可持续性**：没有出现急剧加速或减速，符合健康扩张的特征

### 3. **潜在风险点**
- 2025年前三季度增速（5.2%）略低于第一季度（5.4%），需要关注是否出现增长放缓趋势
- 需要结合其他指标（如PMI、工业增加值、消费数据等）来全面判断经济动能

### 4. **政策建议**
- **扩张期管理**：应关注通胀压力和资产价格泡沫
- **结构性调整**：利用增长窗口期推进经济结构优化
- **风险防范**：为可能的周期转换做好准备

## 关于PMI数据的补充说明

虽然我无法直接获取PMI数据，但从经济周期理论来看：
- **PMI与GDP的关系**：PMI通常领先于GDP，是经济周期的先行指标
- **综合分析的重要性**：理想的经济分析需要结合PMI（制造业景气度）、GDP（总量增长）、就业、通胀等多维度数据
- **周期判断**：PMI持续高于50荣

## 任务3：完整测试（10个场景）

测试SubAgent的综合分析能力，评估输出质量

In [18]:
# 10个测试场景（按难度递增）
test_questions = [
    # 基础数据查询（简单）
    "2024年GDP增长率是多少？",
    "当前的通胀水平如何？",
    "最新的PMI数据是多少？",

    # 周期判断（中等）
    "当前经济处于什么周期？",
    "经济周期转折的信号是什么？",
    "什么指标变化会预示周期转折？",

    # 投资策略（困难）
    "根据当前经济周期，应该配置什么资产？",
    "投资时钟当前处于哪个阶段？",

    # 综合分析（最难）
    "给出当前宏观经济的整体判断",
    "从宏观角度看，周期性行业投资机会如何？"
]

print(f"共{len(test_questions)}个测试问题，按难度递增")
for i, q in enumerate(test_questions, 1):
    print(f"{i}. {q}")

共10个测试问题，按难度递增
1. 2024年GDP增长率是多少？
2. 当前的通胀水平如何？
3. 最新的PMI数据是多少？
4. 当前经济处于什么周期？
5. 经济周期转折的信号是什么？
6. 什么指标变化会预示周期转折？
7. 根据当前经济周期，应该配置什么资产？
8. 投资时钟当前处于哪个阶段？
9. 给出当前宏观经济的整体判断
10. 从宏观角度看，周期性行业投资机会如何？


In [19]:
# 批量测试（注释状态，需要时取消下面的三引号运行）
"""
results = []

for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*60}")
    print(f"测试 {i}/{len(test_questions)}: {question}")
    print('='*60)

    try:
        result = agent.invoke({"messages": [{"role": "user", "content": question}]})
        response_text = result["messages"][-1].content
        results.append({
            'question': question,
            'response': response_text,
            'status': 'success'
        })
        print(response_text)
    except Exception as e:
        results.append({
            'question': question,
            'error': str(e),
            'status': 'failed'
        })
        print(f"[错误] 错误: {e}")

# 保存测试结果
import json
with open('../data/stage2_test_results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\n[完成] 测试完成！成功: {sum(1 for r in results if r['status']=='success')}/{len(results)}")
"""

print("测试脚本已准备，取消三引号可运行")

测试脚本已准备，取消三引号可运行


## 输出质量评估标准

**数据准确性**（30分）：
- [必须] 引用正确来源（"根据AKShare最新数据...）
- [必须] 数据时间明确（"2024年11月..."）
- [必须] 数值准确无误

**分析专业性**（40分）：
- [必须] 运用理论框架（"根据经济周期理论..."）
- [必须] 分析逻辑清晰（数据→趋势→原因→影响）
- [必须] 结合知识库内容

**结论清晰度**（30分）：
- [必须] 给出明确判断（"当前处于XX周期"）
- [必须] 提出可行建议（"建议配置XX资产"）
- [必须] 易于理解（非专业人士能看懂）

**总分≥80分：优秀** | **60-79分：良好** | **<60分：需优化**